In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import md

# 3.1 Harmonic Oscillator

1. Wave number is a measure of cycles per unit distance. I.e. number of waves per unit distance. Wave number $\nu$ is defined as,
\begin{equation}
    \nu = \frac{1}{\lambda},
\end{equation}
with $\lambda$ meaning wavelength, usually in cm. With this equation we can find the conversion to frequency, $f$, angular frequency, $w$, and vibration period $T$.
\begin{align}
    f &= c \nu \\
    w &= \frac{c \nu}{2 \pi} \\
    T &= \frac{1}{c \nu}
\end{align}

In [ ]:
def wavenumber_to_f_w_T(wavenumber):
    """Give wavenumber in units of (cm)^{-1.}"""
    c = 29979245800 # cm/s
    f = c * wavenumber
    w = f / (2 * np.pi)
    T = 1/f
    return f, w, T

In [ ]:
wavenumbers = {
    'c-h': 3000,
    'c-d': 2100,
    'c-c': 1000,
    'c=c': 1700,
    'c#c': 2200
}

In [ ]:
angular_freqs = {}
freqs = {}
periods = {}

for key, wavenumber in wavenumbers.items():
    f, w, T = wavenumber_to_f_w_T(wavenumber)
    freqs[key] = f
    angular_freqs[key] = w
    periods[key] = T
    print(f"{key}: \t f = {f:0.2E} s^{-1}, \n"
          f"\t w = {w:0.2E} s^{-1}, \n"
          f"\t T = {T:0.2E} s")

2. Reduced mass is defined as, 
\begin{equation}
    \mu = \frac{m_{1}m_{2}}{m_1 + m_2}.
\end{equation}
The motion of two bonded particles can be redefined as an oscilation of a single particle with a reduced mass. That single particle oscilates around a characteristic bond length. In our system, this implies that the differential equation defining the behavior is,
\begin{equation}
    \mu \ddot{r} = -kr.
\end{equation}
This implies that the angular frequency, $w$, is related to $k$ by,
\begin{equation}
    w = \sqrt{\frac{k}{\mu}}.
\end{equation}

In [ ]:
def reduce_mass(x, y):
    return x*y/(x+y)

def spring_constant(w, mu):
    return mu * (w)**2

In [ ]:
kg_per_u = 1.661E-27

atm_ms = {
    'c': 12.0107 * kg_per_u,
    'd': 2.0136 * kg_per_u,
    'h': 1.0078 * kg_per_u
}

reduced_masses = {
    'c-h': reduce_mass(atm_ms['c'], atm_ms['h']),
    'c-d': reduce_mass(atm_ms['c'], atm_ms['d']),
    'c-c': reduce_mass(atm_ms['c'], atm_ms['c']),
    'c=c': reduce_mass(atm_ms['c'], atm_ms['c']),
    'c#c': reduce_mass(atm_ms['c'], atm_ms['c'])
}

In [ ]:
spring_constants = {}
for key, redu_m in reduced_masses.items():
    k = spring_constant(angular_freqs[key], redu_m)
    spring_constants[key] = k
    print(f"{key}: k = {k:0.3f}")

In [ ]:
df = pd.DataFrame([wavenumbers, freqs, angular_freqs, periods, spring_constants], 
                  index=['wavenumber', 'frequency', 'angular frequency', 'period', 'spring constant'])

In [ ]:
df

3. As seen in the table above, the relationship between chemical bond strength is ...

# 3.2 Integration of the equation of motion

1. Blah blah blah...

2. Blah blah blah...

3. Blah blah blah...

4. Blah blah blah...

5. Blah blah blah...

6. Blah blah blah...